# Gemini with Pydantic-AI

This notebook demonstrates how to build a simple conversational agent using Google's Gemini models and the `pydantic-ai` library. The agent can use tools, like fetching weather information, based on the conversation context.

Learn more about `pydantic-ai` here: [https://github.com/pydantic-ai/pydantic-ai](https://github.com/pydantic-ai/pydantic-ai)

In [ ]:
%pip install pydantic-ai

In [2]:
from pydantic_ai import Agent
from geopy.geocoders import Nominatim
import requests

geolocator = Nominatim(user_agent="weather-app") 

weather = Agent('google-gla:gemini-2.0-flash')


@weather.tool_plain(docstring_format='google', require_parameter_descriptions=True)
async def get_weather_forecast(location: str, date: str):
    """Retrieves the weather using Open-Meteo API for a given location (city) and a date (yyyy-mm-dd). Returns a list dictionary with the time and temperature for each hour.
    
    Args:
        location: The city and state, e.g., San Francisco
        date: the forecasting date for when to get the weather format (yyyy-mm-dd)
    """
    
    location = geolocator.geocode(location)
    if location:
        try:
            response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={location.latitude}&longitude={location.longitude}&hourly=temperature_2m&start_date={date}&end_date={date}")
            data = response.json()
            return {time: temp for time, temp in zip(data["hourly"]["time"], data["hourly"]["temperature_2m"])}
        except Exception as e:
            return {"error": str(e)}
    else:
        return {"error": "Location not found"}
    
# res_1 = await weather.run("What is the weather in Berlin on 12th of March 2025?")
res_1 = await weather.run("How is the weather in Berlin on 12th of March 2025?")
# res_2 = await weather.run("Would it be warmer in Munich?", message_history=res_1.new_messages())
res_2 = await weather.run("Would it be in Munich warmer?", message_history=res_1.new_messages())

res_2

AgentRunResult(data='Yes, it looks like it will be warmer in Munich on March 12, 2025. The temperature in Munich will range from 3-6 degrees Celsius in the early morning, increasing to 9-10 degrees Celsius during the afternoon, and then decreasing to 6-9 degrees Celsius in the evening.\n')